## Assignment 4 : web scraping 
#### Name : Amit Gido
#### ID : 315754606

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re


In [2]:
URL = "https://www.madlan.co.il/for-sale/הוד-השרון-ישראל"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}
page = requests.get(URL, headers=headers)


In [6]:
if page.status_code == 200:
    soup = BeautifulSoup(page.content, "html.parser")
    apartments = []
    links = soup.find_all('a', href=True)
    for link in links:
        href = link['href']
        if href.startswith('/listings'):
            apartment_url = f"https://www.madlan.co.il{href}"
            print(apartment_url)
            apartment_page = requests.get(apartment_url, headers=headers)
            if apartment_page.status_code == 200:
                apartment_soup = BeautifulSoup(apartment_page.content, "html.parser")

                primary_address = apartment_soup.find('h1', {'data-auto': 'primary_address_text'})
                if primary_address is None:
                    continue

                address_parts = primary_address.text.split()
                if len(address_parts) < 2:
                    continue

                street = ' '.join(address_parts[:-1])
                number_in_street = ''.join(filter(str.isdigit, address_parts[-1]))
                if number_in_street:
                    number_in_street = int(number_in_street)
                else:
                    number_in_street = None

                secondary_address = apartment_soup.find('div', {'data-auto': 'secondary_address_text'}).text
                if ',' in secondary_address:
                    city_area, city = secondary_address.split(', ')
                else:
                    city_area = ''
                    city = secondary_address

                type_ = apartment_soup.find('div', {'data-auto': 'business-class'}).text
                room_number = apartment_soup.find('div', {'data-auto': 'beds-count'}).text

                floor_out_of = apartment_soup.find('div', {'data-auto': 'floor'}).text
                area = apartment_soup.find('div', {'data-auto': 'area'}).text

                condition = apartment_soup.find('div', {'data-auto': 'unit-general-condition-value'}).text
                entranceDate = apartment_soup.find('div', {'data-auto': 'unit-availability-value'}).text
                furniture = apartment_soup.find('div', {'data-auto': 'unit-furniture-details-value'}).text

                description_elem = apartment_soup.find('div', {'class': 'css-v1qjdi ebqee3y2'})
                description = description_elem.text if description_elem else ''

                price_text = apartment_soup.find('div', {'data-auto': 'current-price'}).text
                price = int(re.sub(r'\D', '', price_text))
                specification = apartment_soup.find('div', {'data-auto' : "amenities-block"}).get_text()
                hasElevator = 'יש מעלית' in specification
                hasParking = "יש חניה" in specification
                hasBars = "יש סורגים" in specification
                hasStorage = 'יש מחסן' in specification
                hasAirCondition = "יש מיזוג אויר" in specification
                hasBalcony = "יש מרפסת" in specification
                hasMamad = "יש ממ" in specification
                handicapFriendly = "לא נגיש לנכים" not in specification
                num_of_images =len(apartment_soup.find_all('div' ,{'data-auto':"bulletin-image"}))
                

                apartments.append({
                    'City': city,
                    'type': type_,
                    'room_number': room_number,
                    'Area': area,
                    'Street': street,
                    'number_in_street': number_in_street,
                    'city_area': city_area,
                    'price': price,
                    'floor_out_of': floor_out_of,
                    'condition': condition,
                    'entranceDate': entranceDate,
                    'furniture': furniture,
                    'description': description,
                    'hasElevator': hasElevator,
                    'hasParking' : hasParking,
                    'hasBars' : hasBars,
                    'hasStorage' : hasStorage,
                    'hasAirCondition' : hasAirCondition,
                    'hasBalcony' : hasBalcony,
                    'hasMamad' : hasMamad,
                    'handicapFriendly' : handicapFriendly,
                    'num_of_images': num_of_images })
                
            else:
                print(f'Error status code {apartment_page.status_code} link {apartment_url}')
                break
                

    df = pd.DataFrame(apartments)
    df
    
else:
    print(f'Error status code {page.status_code}')
    

Error status code 403


In [4]:
df.to_excel('real_estate_output.xlsx')


NameError: name 'df' is not defined

In [ ]:
df
